In [1]:
import datetime as dt
import time
import warnings
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import ruptures as rpt

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.optimizers import Adam, Adamax, RMSprop
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [2]:
sns.set_style('whitegrid')
colors = sns.color_palette('deep', 8)
sns.set_palette(colors)

In [3]:
df_orig = pd.read_json("../datasets/df.json", orient='split')
df = df_orig.copy()
df = df[df.eth_close.isnull().sum():]
df.fillna(0, inplace=True)
df.set_index("date", drop=True, inplace=True)
print(df.shape)
df.head(2)

(1134, 15)


,eth_open,eth_high,eth_low,eth_close,eth_volumefrom,eth_volumeto,btc,xrp,eos,ltc,xlm,xmr,vixcls,twexb,effr
date,,,,,,,,,,,,,,,
2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79,277.89,0.008013,0.0,4.10,0.0,0.75,13.77,118.1833,0.14
2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16,258.60,0.009000,0.0,3.77,0.0,0.75,13.39,118.1833,0.14


## Feature Engineering
### Create Dummies for Changepoints

In [ ]:
cpts = rpt.Pelt(model="rbf").fit_predict(pd.DataFrame(df.eth_close), pen=20)
print(cpts)

In [ ]:
breaks = cpts[:-1]
for i, point in enumerate(breaks):
    df["break_{}".format(i)] = 0
    df["break_{}".format(i)][point:] = 1

## Define Functions

In [ ]:
def prep_data(data, window_size = 1):
    data_reshaped = np.reshape(data, (data.shape[0], window_size, data.shape[1]))
    return data_reshaped

def fit_lstm(X_train, y_train, epochs=500, optimizer=Adam(lr=0.005),):
    model = Sequential()
    model.add(LSTM(units=1,
                   activation='tanh',
                   use_bias=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    return model

def validate(model, X, y):
    yhat = model.predict(X)
    yhat_invert = scaler.inverse_transform(yhat)
    y_orig = scaler.inverse_transform(y)
    rmse = np.sqrt(mean_squared_error(y_orig, yhat_invert))
    return (yhat_invert, y_orig, rmse)

## LSTM RNN Time Series Only

In [ ]:
# set time series data & target
X = df.eth_close
y = df.shift(-1).eth_close

# split into train/test
X_train, X_test = X[:-31], X[-31:-1]
y_train, y_test = y[:-31], y[-31:-1]

# reshape & scale
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

scaler = MinMaxScaler(feature_range=(0,1))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.fit_transform(y_test)

# prep for NN
X_train = prep_data(X_train)
X_test = prep_data(X_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
lstm_ts = fit_lstm(X_train, y_train)

In [ ]:
# validate results
train_val = validate(lstm_ts, X_train, y_train)
test_val = validate(lstm_ts, X_test, y_test)

print("train rmse = {:.5f}".format(train_val[2]))
print("test rmse = {:.5f}".format(test_val[2]))

**Best model params:**

(LSTM(1, activation='tanh', use_bias=True, input_shape=(X_train.shape[1], X_train.shape[2])))

Loss: mean squared error
Optimizer: Adam
Learning Rate: 0.005
Epochs: 500
    
RMSE 15.16xxx - 15.18xxx

In [ ]:
plt.figure(figsize=(11,6))

# observed
plt.plot(X.index[-31:-1],
         test_val[1],
         label="observed",
         c=colors[0])
# predictions
plt.plot(X.index[-31:-1],
         test_val[0],
         label="test set predictions\nrmse = {:.5f}".format(test_val[2]),
         c=colors[2])
plt.legend()
plt.show()

## LSTM TS only, include break point dummies

In [ ]:
# set time series data & target
X = df.loc[:, df.columns.isin(["eth_close",
                               "break_0", "break_1", "break_2", "break_3"])]
y = df.shift(-1).eth_close

# split into train/test
X_train, X_test = X[:-31], X[-31:-1]
y_train, y_test = y[:-31], y[-31:-1]

# reshape and scale
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

scaler = MinMaxScaler(feature_range=(0,1))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.fit_transform(y_test)

# prep for NN
X_train = prep_data(X_train)
X_test = prep_data(X_test)
print(X_train.shape, X_test.shape)

In [ ]:
lstm_cpts = fit_lstm(X_train, y_train)

In [ ]:
train_cpts_val = validate(lstm_cpts, X_train, y_train)
test_cpts_val = validate(lstm_cpts, X_test, y_test)

print("train rmse = {:.5f}".format(train_cpts_val[2]))
print("test rmse = {:.5f}".format(test_cpts_val[2]))

## LSTM RNN w/ Exogenous Variables

In [ ]:
# set time series data & target
X = df
y = df.shift(-1).eth_close

X_train, X_test = X[:-31], X[-31:-1]
y_train, y_test = y[:-31], y[-31:-1]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# reshape and scale
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

scaler = MinMaxScaler(feature_range=(0,1))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.fit_transform(y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
X_train = prep_data(X_train)
X_test = prep_data(X_test)
print(X_train.shape, X_test.shape)

In [ ]:
# train model
lstm_exog = fit_lstm(X_train, y_train)

In [ ]:
# validate results
trainx_val = validate(lstm_exog, X_train, y_train)
testx_val = validate(lstm_exog, X_test, y_test)

print("train rmse = {:.5f}".format(trainx_val[2]))
print("test rmse = {:.5f}".format(testx_val[2]))

In [ ]:
plt.figure(figsize=(11,6))

# observed
plt.plot(X.index[-31:-1],
         test_val[1],
         label="observed",
         c=colors[0])
# ts predictions
plt.plot(X.index[-31:-1],
         test_val[0],
         label="ts test set\nrmse = {:.5f}".format(test_val[2]),
         c=colors[2])
# exog predictions
plt.plot(X.index[-31:-1],
         testx_val[0],
         label="exog test set\nrmse = {:.5f}".format(testx_val[2]),
         c=colors[3])
plt.legend()
plt.show()